In [1]:
import pandas as pd
import os
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from datetime import datetime
import time
from sklearn.metrics import f1_score, confusion_matrix

In [2]:
DATA_PATH = '../../Pokemon-data/'
NORMALIZED_DATA = '../../Training-baseline/'
SEED = 42
BATCH_SIZE = 64
EPOCHS = 1
PRETRAINED = True
MODEL_NAME = str(EPOCHS) + "Epochs-freeze"
checkpoint_path = './saved-models/'
n_epochs_stop = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
metadata = pd.read_csv('metadata/pokemon.csv')

pokemon_names = [x for x in os.listdir(NORMALIZED_DATA)]

filtered_metadata = metadata[metadata.name.isin(pokemon_names)]

filtered_list = filtered_metadata.loc[:,'name'].tolist()

filtered_metadata = filtered_metadata.loc[:,['name', 'type1']]

#fire_data = filtered_metadata[filtered_metadata["type1"] == 'fire']
#grass_data = filtered_metadata[filtered_metadata["type1"] == 'grass']
#filtered_metadata = pd.concat([fire_data, grass_data], ignore_index=True)

filtered_metadata.type1 = pd.Categorical(filtered_metadata.type1)
filtered_metadata['code'] = filtered_metadata.type1.cat.codes
print(filtered_metadata['code'])

def compile_training_data_to_list():
    all_data = []
    for pokemon in os.listdir(NORMALIZED_DATA):
        all_data += [pokemon + '/' + x for x in os.listdir(NORMALIZED_DATA + pokemon)]

    results = create_annotated_dataframe(all_data)
    return results

def create_annotated_dataframe(all_data):
    base_data = {'file_name': [], 'name': [], 'label': []}
    for item in all_data:
        if len(filtered_metadata[filtered_metadata['name'].str.contains(item.split('/')[0])]):
            base_data['file_name'].append(item)
            base_data['name'].append(item.split('/')[0])
            # yes, this is a bit ugly, but we have to match with the metadata
            base_data['label'].append(
                filtered_metadata[
                    filtered_metadata['name']==(item.split('/')[0])
                ].loc[:,'code'].tolist()[0])

    results, y_train = create_encoded_dataframe(base_data)
    return results, y_train

def create_encoded_dataframe(base_data):
    results = pd.DataFrame(base_data, columns = ['file_name', 'name', 'label'])
    labels = np.unique(results["label"])
    max_value = np.max(labels) + 1
    Y_train = np.eye(max_value)[results["label"]]
    return results, Y_train

0       9
1       9
2       9
3       6
4       6
       ..
714     7
715     4
716     1
718    15
720     6
Name: code, Length: 703, dtype: int8


In [4]:
encoded_data, y_train = compile_training_data_to_list()

In [5]:
class CustomDataset(Dataset):
	def __init__(self, x, y, img_dir):
		self.x = x
		self.y = y
		self.img_dir = img_dir
		self.classes = np.unique(self.y)


	def __len__(self):
		return len(self.x)


	def __getitem__(self, idx):
		img_path = os.path.join(self.img_dir, self.x[idx])
        # use the slice to remove a possible 4th alpha channel
		#image = iio.v2.imread(img_path)[:,:,:3]
		image = np.load(img_path)[:,:,:3]
		image = image.astype(np.float32)
		label = self.y[idx]
		return image, label

In [6]:
def stratified_split(dataset, labels):
    x_train, x_val, y_train, y_val = train_test_split(dataset['file_name'].to_numpy(),
                                                  labels,
                                                  test_size=0.25,
                                                  stratify=dataset['label'],
                                                  random_state=SEED)

    trainSet = CustomDataset(x_train, y_train, NORMALIZED_DATA)
    valSet = CustomDataset(x_val, y_val, NORMALIZED_DATA)
    return trainSet, valSet

In [7]:
def print_performance_metrics(currentEpoch, avgTrainLoss, trainCorrect, avgValLoss, valCorrect, trainf1, valf1):
    print()
    print(f"[INFO] EPOCH: {currentEpoch + 1}/{EPOCHS}")
    print(f"Train loss: {avgTrainLoss :.6f}, Train accuracy: {trainCorrect :.4f}")
    print(f"Val loss: 	{avgValLoss :.6f}, Val accuracy:   {valCorrect :.4f}")
    print(f"Train F1: 	{trainf1 :.6f}")
    print(f"Val F1: 	{valf1 :.6f}")
    print(f"---------------------------------------------------- \n")

In [8]:
train, val = stratified_split(encoded_data, y_train)
trainDataLoader = DataLoader(train, batch_size=BATCH_SIZE)
valDataLoader = DataLoader(val, batch_size=BATCH_SIZE)

trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [9]:
from torchvision import models
model = models.efficientnet_b0(pretrained=True)

In [10]:
for param in model.parameters():
    param.requires_grad = False

In [11]:
import torch.nn as nn
# Add on classifier
model.classifier = nn.Sequential(
                      nn.Linear(1280, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, 18),                   
                      nn.LogSoftmax(dim=1))

In [12]:
model = model.to('cuda')

In [13]:
torch.manual_seed(SEED)
class_acc = []
epochs_no_improve = 0
metrics = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": [],
	"trainf1": [],
    "valf1": []
}

In [14]:
from torch import optim
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [15]:
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

for epoch in range(EPOCHS):
    print(f"[INFO] starting training for epoch: {epoch + 1} at {datetime.now().strftime('%H:%M:%S')}")

    train_preds = []
    train_target = []

    val_preds = []
    val_target = []

    model.train()

    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    trainCorrect = 0
    valCorrect = 0
    min_val_loss = 100

    for data, targets in tqdm(trainDataLoader, desc="Training progress"):
        data = data.transpose(1, 3)
        data = data.transpose(2, 3)

        data = data.float()
        data, targets = (data.to(device), targets.to(device))
        # Generate predictions
        out = model(data)
        # Calculate loss
        loss = criterion(out, targets)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss

        for item in out:
            train_preds.append(np.argmax(item.cpu().detach().numpy()))
            trainCorrect += (item.argmax(0) == targets.argmax(0)).type(torch.float).sum().item()

        for label in targets:
            train_target.append(label.cpu().detach().numpy().argmax(0))

    with torch.no_grad():
        model.eval()
        print(f"[INFO] starting evaluation for epoch: {epoch + 1} at {datetime.now().strftime('%H:%M:%S')}")
        for data, targets in tqdm(valDataLoader, desc="Validation progress"):
            data = data.transpose(1, 3)
            data = data.transpose(2, 3)

            data = data.float()
            data, targets = (data.to(device), targets.to(device))
            # Generate predictions
            out = model(data)
            # Calculate loss
            totalValLoss += criterion(out, targets)
            for item in out:
                val_preds.append(np.argmax(item.cpu().detach().numpy()))
                valCorrect += (item.argmax(0) == targets.argmax(0)).type(torch.float).sum().item()

            for label in targets:
                val_target.append(label.cpu().detach().numpy().argmax(0))

        # Average validation loss
        avg_val_loss = totalValLoss / valSteps
        # If the validation loss is at a minimum
        if avg_val_loss < min_val_loss:
            # Save the model
            torch.save(model, checkpoint_path + MODEL_NAME + ".pt")
            epochs_no_improve = 0
            min_val_loss = avg_val_loss

        else:
            epochs_no_improve += 1
            # Check early stopping condition
            if epochs_no_improve == n_epochs_stop:
                print('Early stopping!')

    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps

    # calculate per class accuracy via confusion matrix
    matrix = confusion_matrix(val_target, val_preds)
    class_acc.append(matrix.diagonal() / matrix.sum(axis=1))

    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(trainDataLoader.dataset)
    valCorrect = valCorrect / len(valDataLoader.dataset)

    # calculate macro F1 score
    trainf1 = f1_score(train_target, train_preds, average='macro')
    valf1 = f1_score(val_target, val_preds, average="macro")

    # update our training history
    metrics["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    metrics["train_acc"].append(trainCorrect)
    metrics["val_loss"].append(avg_val_loss.cpu().detach().numpy())
    metrics["val_acc"].append(valCorrect)
    metrics['trainf1'].append(trainf1)
    metrics['valf1'].append(valf1)

    # print the model training and validation information
    print_performance_metrics(epoch, avgTrainLoss, trainCorrect, avg_val_loss, valCorrect, trainf1, valf1)

endTime = time.time()
print(f"[INFO] total time taken to train the model: {(endTime - startTime) / 60 :.2f}min")

[INFO] training the network...
[INFO] starting training for epoch: 1 at 14:37:35


Training progress: 100%|██████████| 142/142 [03:55<00:00,  1.66s/it]


[INFO] starting evaluation for epoch: 1 at 14:41:31


Validation progress: 100%|██████████| 48/48 [00:35<00:00,  1.35it/s]



[INFO] EPOCH: 1/1
Train loss: 2.614307, Train accuracy: 0.6894
Val loss: 	2.395915, Val accuracy:   0.7828
Train F1: 	0.142272
Val F1: 	0.233775
---------------------------------------------------- 

[INFO] total time taken to train the model: 4.53min
